# Update spotify net/assign tracks

> Update spotify net by assigning tracks with above a playcount threshold to an existing public playlist and removing from net

In [1]:
#| default_exp prep_model

In [2]:
#| export
import boto3
import numpy as np
import pandas as pd
import pickle

In [3]:
#hide
from nbdev.showdoc import *

In [4]:
#| export
class ModelPrep:
    S3_BUCKET = 'spotify-net'
    SCALER_FILE = 'scaler'
    SVD_FILE = 'svd'
    SPOTIFY_TRACKS_FILE = 'newer_tracks.csv'
    LASTFM_TRACKS_FILE = 'last_fm_recent_tracks.csv'
    GENRES_SVD_FILE = 'genres_svd.csv'
    KEY_LIST_FILE = 'key_list.csv'
    TIMESIG_LIST_FILE = 'timeSig_list.csv'
    
    def __init__(self):
        self.scaler = None
        self.svd = None
        self.prepped_frame = None
        self.genre_series = None
        self.key_series = None
        self.time_signature_series = None
        self.s3_resource = boto3.resource('s3')
    
    def load_scaler(self):
        scaler = self.s3_resource.Object(self.S3_BUCKET, self.SCALER_FILE).get()
        self.scaler = pickle.loads(scaler['Body'].read())

    def load_svd(self):
        svd = self.s3_resource.Object(self.S3_BUCKET, self.SVD_FILE).get()
        self.svd = pickle.loads(svd['Body'].read())

    def load_tracks_data(self):
        spotify_tracks = pd.read_csv(f's3://{self.S3_BUCKET}/{self.SPOTIFY_TRACKS_FILE}', index_col=0)
        lastFM_tracks = pd.read_csv(f's3://{self.S3_BUCKET}/{self.LASTFM_TRACKS_FILE}', index_col=0)
        spotify_tracks[['name', 'artist']] = spotify_tracks[['name', 'artist']].applymap(str.upper)
        lastFM_tracks[['name', 'artist']] = lastFM_tracks[['name', 'artist']].applymap(str.upper)
        self.prepped_frame = pd.merge(spotify_tracks, lastFM_tracks, on=['name', 'artist'])
        self.prepped_frame = self.prepped_frame.drop([
            'playcount', 
            'added at', 
            'artist id',
            'id',
            'type',
            'track_href',
            'analysis_url',
            'type',
            'diff',
            'uri'
        ], axis=1)
    
    def load_genre_series(self):
        genre_series = pd.read_csv(f's3://{self.S3_BUCKET}/{self.GENRES_SVD_FILE}', index_col=0, squeeze=True)
        self.genre_series = genre_series

    def load_key_series(self):
        key_series = pd.read_csv(f's3://{self.S3_BUCKET}/{self.KEY_LIST_FILE}', index_col=0, squeeze=True)
        self.key_series = key_series

    def load_time_signature_series(self):
        time_signature_series = pd.read_csv(f's3://{self.S3_BUCKET}/{self.TIMESIG_LIST_FILE}', index_col=0, squeeze=True)
        self.time_signature_series = time_signature_series

    # Load genre, key, and time signature data. How is this list of genres being generated?
    # These don't need to be loaded from S3, but I'm doing it anyway because...?     
    def load_s3(self):
        self.load_scaler()
        self.load_svd()
        self.load_tracks_data()
        self.load_genre_series()
        self.load_key_series()
        self.load_time_signature_series()
    
    # TODO: should prepped frame be an argument? Why not an instance variable?
    def transform_features(self, constant):
        # ok, so order I'm log transforming before I'm applying the standard scaler later on? What is the interaction between these two operations generally?
        # also, review feature selection process as it relates to feature transformation, and traditional order here

        # Log-transform 'speechiness', 'acousticness', 'instrumentalness'
        self.prepped_frame[['speechiness', 'acousticness', 'instrumentalness']] += constant
        self.prepped_frame[['speechiness', 'acousticness', 'instrumentalness']] = np.log(self.prepped_frame[['speechiness', 'acousticness', 'instrumentalness']])

        # One-hot encode 'key' and 'time_signature'
        self.prepped_frame = pd.get_dummies(self.prepped_frame , prefix=['key', 'time_signature'], columns=['key', 'time_signature'])

        # apply standardScaler to 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms':
        columns_to_scale = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
        self.prepped_frame[columns_to_scale] = self.scaler.transform(self.prepped_frame[columns_to_scale])
    
    def update_dataframe_with_prefix(self, current_dataframe, series, prefix):
        current_columns = current_dataframe.loc[:, current_dataframe.columns.str.startswith(prefix)]
        all_columns = pd.DataFrame(np.zeros((len(current_dataframe), len(series))) , columns=series.tolist())
        all_columns.update(current_columns)
        current_dataframe = current_dataframe.loc[:, ~current_dataframe.columns.str.startswith(prefix)]

        return pd.concat([current_dataframe, all_columns], axis=1)

    def prepare_final_frame(self):
        # One-hot encode genres and reduce to top 60 components using SVD
        current_genres = self.prepped_frame.loc[:, self.prepped_frame.columns.str.startswith('genre_')]
        all_genres = pd.DataFrame(np.zeros((len(self.prepped_frame), len(self.genre_series))) , columns=self.genre_series.tolist())
        all_genres = all_genres.add_prefix('genre_')
        all_genres.update(current_genres)
        all_genres.columns = all_genres.columns.str.replace('genre_', '')

        # Reduce features to top 60 components
        transformed_genres = pd.DataFrame(self.svd.transform(all_genres))
        transformed_genres = transformed_genres.add_prefix('genre_')

        self.prepped_frame = self.prepped_frame.loc[:, ~self.prepped_frame.columns.str.startswith('genre_')]
        self.prepped_frame = pd.concat([self.prepped_frame, transformed_genres], axis=1)

        # One-hot encode keys and time signatures
        self.prepped_frame = self.update_dataframe_with_prefix(self.prepped_frame, self.key_series, 'key_')
        self.prepped_frame = self.update_dataframe_with_prefix(self.prepped_frame, self.time_signature_series, 'time_signature_')

    def save_prepared_frame(self):
        self.prepped_frame.to_csv(f's3://{self.S3_BUCKET}/for_prediction.csv')
        print(self.prepped_frame.shape)
        print('Uploaded to S3')

In [5]:
#| export
if __name__ == '__main__':
    frame = ModelPrep()
    frame.load_s3()
    frame.transform_features(0.0000001)
    frame.prepare_final_frame()
    frame.save_prepared_frame()

(2, 90)
Uploaded to S3
